# Hidden Markov: POS Tagging
Sam Keyser, Carter Shavitz, John Paul Bunn

CS 2400 - Introduction to AI

## Experiment
### Set Up

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.util import ngrams
from nltk.corpus import brown, treebank, conll2000

# Download the requisite datasets
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')
nltk.download('universal_tagset')

# Load datasets
treebank_corpus = treebank.tagged_sents(tagset='universal')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\keysers\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\keysers\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\keysers\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\keysers\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [12]:
print(treebank_corpus)

# Get a test X, y out of the corpus
X, y = zip(*treebank_corpus[0])
X = list(X)
y = list(y)
print('Sentence:', X)
print('Tags:', y)

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]
Sentence: ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
Tags: ['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


## Probability Counting
Now that we've got a set of test sentences and tags, we need to start constructing the transition and emission probabilities. This count should be a function *N*, which is the length of the *N*-gram which we use to keep track of previous states up to the current one.

### Playing around with Splitting Sentences into *N*-grams

In [17]:
N = 2 # Default N-gram length

Example of splitting using ngram from nltk

In [19]:
print(X)
print(*ngrams(X, N)) # Split up our X

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
('Pierre', 'Vinken') ('Vinken', ',') (',', '61') ('61', 'years') ('years', 'old') ('old', ',') (',', 'will') ('will', 'join') ('join', 'the') ('the', 'board') ('board', 'as') ('as', 'a') ('a', 'nonexecutive') ('nonexecutive', 'director') ('director', 'Nov.') ('Nov.', '29') ('29', '.')


In [6]:
### Playing around with Conditional Probability

Counting probability based on the article [here](https://www.freecodecamp.org/news/a-deep-dive-into-part-of-speech-tagging-using-viterbi-algorithm-17c8de32e8bc).

In [54]:
N = 2
X = ['Noise', 'Quiet', 'Quiet', 'Noise']
y = ['Awake', 'Awake', 'Asleep', 'Awake']

# pre/append start/end
X.insert(0, '*')
X.append('*')

y.insert(0, '.')
y.append('.')


states = set(y) # all the states we can see
print('states:', states)

obs = X
print('obs:', X)
print('tags:', y)

c = {} # counts
for observation, state in zip(X, y):
    c[state] = c.get(state, 0) + 1
    c[(observation, state)] = c.get((observation, state), 0) + 1

e = {} # emission probabilities
for observation, state in zip(X, y):
    e[(observation, state)] = c[(observation, state)] / c[state]

print('\ncounts')
print(c)
print('\nemission probabilities')
print(e)

state_c = {}
gram_c = {} # gram counts
q = {} # transition probabilities
# for gram in ngrams(y, N):
#     gram_c[gram] = gram_c.get(gram, 0) + 1
#
# for state in y:
#     state_c[state] = state_c.get(state, 0) + 1

print()
for gram in ngrams(y, N):
    print(gram)
    q[gram[-1]] = q.get(gram[-1], 0) + 1
    q[gram] = q.get(gram, 0) + 1
    q[gram[:-1]] = q.get(gram[:-1], 0) + 1

for gram in q.keys():
    if len(gram) == N:
        q[gram] = q[gram]/q[gram[:-1]]


# print('\nngram counts')
# print(gram_c)
# print('\nstate counts')
# print(state_c)
print('\ntransition probabilities')
print(q)

states: {'Awake', '.', 'Asleep'}
obs: ['*', 'Noise', 'Quiet', 'Quiet', 'Noise', '*']
tags: ['.', 'Awake', 'Awake', 'Asleep', 'Awake', '.']

counts
{'.': 2, ('*', '.'): 2, 'Awake': 3, ('Noise', 'Awake'): 2, ('Quiet', 'Awake'): 1, 'Asleep': 1, ('Quiet', 'Asleep'): 1}

emission probabilities
{('*', '.'): 1.0, ('Noise', 'Awake'): 0.6666666666666666, ('Quiet', 'Awake'): 0.3333333333333333, ('Quiet', 'Asleep'): 1.0}

('.', 'Awake')
('Awake', 'Awake')
('Awake', 'Asleep')
('Asleep', 'Awake')
('Awake', '.')

transition probabilities
{'Awake': 3, ('.', 'Awake'): 1.0, ('.',): 1, ('Awake', 'Awake'): 0.3333333333333333, ('Awake',): 3, 'Asleep': 1, ('Awake', 'Asleep'): 0.3333333333333333, ('Asleep', 'Awake'): 1.0, ('Asleep',): 1, '.': 1, ('Awake', '.'): 0.3333333333333333}


### More Rigorous Conditional Probability
First, lets reload our test data.

In [10]:
N = 3
#X = [[word for word, tag in sentence] for sentence in treebank_corpus]
#y = [[tag for word, tag in sentence] for sentence in treebank_corpus]
X = treebank_corpus
X

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

We write the following conditional probability class based on [this](https://github.com/edorado93/HMM-Part-of-Speech-Tagger/blob/master/hmmlearn.py).

In [11]:
%%file conditional_prob.py
import math
import itertools
import functools

# convenience method
def increment_dict_val(dict, val):
    dict[val] = dict.get(val, 0) + 1

class ConditionalProbability:
    start_tag = '!@#$START$#@!' # Pick some arbitrary string that is not likely to be naturally embedded into our text data

    k = 10 # k fold hyperparameter

    def __init__(self, X, N=3):
        assert(N >= 2)

        # Init dataset and hyperparameters
        self.X = X
        #self.y = y
        self.N = N

        # P(Wi|Ck)
        self.words_given_pos = {}

        # conditional probability of P(Xi+N|Xi+N-1...Xi)
        self.full_ngram = {}

        # maps word to set of tags associated with it
        self.word_to_tag = {}

        # maps (word, tag) pairings to # of appearances
        self.word_tag_count = {}

        # tracks the number of time a tag has appeared
        self.tag_count = {{}}

        # counts the occurrence of n-gram of tags
        self.ngram_counts = {}

        # counts the occurrences of (n-1)-grams of tags
        self.subset_ngram_counts = {}

        # set of all tags we've seen
        self.tags = set()

        # set of all words
        self.words = set()

        ''' BACK-OFF PROB '''
        self.transition_backoff = {}
        self.emission_backoff = {}

        ''' SINGLETON COUNTS '''
        self.transition_singleton = {}
        self.emission_singleton = {}

        ''' 1-COUNT SMOOTHED PROB '''
        self.transition_one_count = {}
        self.emission_smoothed = {}

        self.n = 0 #TODO: what am I

    def calculate_probabilities(self):
        self.populate_dictionaries()
        self.CFD_word_given_tag()
        self.CFD_ngram_tags()
        self.backoff_probabilities()
        self.singleton_counts()
        self.smooth_probabilities()
        #self._save()

    def populate_dictionaries(self):
        self.pos_tags = set()

        start_tag = ConditionalProbability.start_tag
        for sentence in X:
            for iter in range(0, self.N-1): # We need N-1 start characters to support N-grams
                sentence.insert(0, (start_tag, start_tag))

            start_idx = self.N-1
            for idx in range(start_idx, len(sentence)):
                ngram = tuple([sentence[jdx][1] for jdx in range(idx-N-1, idx+1)])
                sub_ngram = ngram[:-1]

                self.ngram_counts[ngram] = self.ngram_counts.get(ngram, 0) + 1
                self.subset_ngram_counts[sub_ngram] = self.subset_ngram_counts.get(sub_ngram, 0) + 1

            for word, tag in sentence:
                self.n += 1

                # do back off counts
                self.transition_backoff[tag] = self.transition_backoff.get(tag, 0) + 1
                self.emission_backoff[word] = self.emission_backoff.get(word, 0) + 1

                self.tags.add(tag)
                self.words.add(word)

                increment_dict_val(self.word_tag_count, (word, tag))
                increment_dict_val(self.tag_count, tag)

                if word not in self.word_to_tag:
                    self.word_to_tag[word] = set()
                self.word_to_tag[word].add(tag)

    def backoff_probabilities(self):
        V = len(self.tags)

        for word in self.emission_backoff:
            self.emission_backoff[word] = float(1 + self.emission_backoff[word]) / float(self.n + V)

        for tag in self.transition_backoff:
            self.transition_backoff[tag] = float(self.transition_backoff[tag]) / float(self.n)

    def singleton_counts(self):
        for permutation in itertools.permutations(self.tags):
            if permutation in self.ngram_counts and self.ngram_counts[permutation] == 1:
                increment_dict_val(self.transition_singleton, permutation[1:])


        for word in self.words:
            for tag in self.tags:
                word_tag = (word, tag)
                if word_tag in self.word_tag_count and self.word_tag_count[word_tag] == 1:
                    increment_dict_val(self.emission_singleton, tag)

    def smooth_probabilities(self):
        start_idx = self.N - 1

        for sentence in self.X:
            for idx in range(start_idx, len(sentence)):
                ngram = tuple([sentence[jdx][1] for jdx in range(idx-N-1, idx+1)])
                sub_ngram = ngram[:-1]
                lamda = 1 + self.transition_singleton.get(sub_ngram, 0)
                self.transition_one_count[ngram] = math.log(float(self.ngram_counts[ngram] + lamda * self.transition_backoff[sentence[idx][1]]) / float(self.subset_ngram_counts[sub_ngram] + lamda))

        for word, tag_set in self.word_to_tag.items():
            for tag in tag_set:
                lamda = 1 + self.emission_singleton.get(tag, 0)
                self.emission_smoothed[(word, tag)] = math.log(float(self.word_tag_count[(word, tag)] + lamda * self.emission_backoff[word]) / float(self.tag_count[tag] + lamda))

    def CFD_word_given_tag(self):
        '''
        P(word|tag)
        :return:
        '''
        for word, tag_set in self.word_to_tag.items():
            for tag in tag_set:
                self.words_given_pos[(word, tag)] = math.log(float(self.word_tag_count[(word, tag)]) / float(self.tag_count[tag]))

    def CFD_ngram_tags(self):
        '''
        probability of a tag s given last n-1 tags
        :return:
        '''
        start_idx = self.N-1
        V = len(self.tags)

        for sentence in X:
            for idx in range(start_idx, len(sentence)):
                ngram = tuple([sentence[jdx][1] for jdx in range(idx-N-1, idx+1)])
                sub_ngram = ngram[:-1]

                self.full_ngram[ngram] = math.log(float(1 + self.ngram_counts[ngram]) / float(V + self.subset_ngram_counts[sub_ngram]))

Writing conditional_prob.py
